<h2>Run posterior sampler on GCMS data</h2>

In [46]:
%load_ext autoreload
%autoreload 2
import sys,os
sys.path.append('../fragments')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
from network_sampler import *
from frag_set import *

In [48]:
gcms_peaks = FragSet()
gcms_peaks.load_from_file('mzMATCHoutput.txt')
gcms_peaks.load_annotations('temp_nist.txt')

Loading from mzMATCHoutput.txt
Loaded 11870 peaks
...with 2272 relations
	Parsing NIST output
	Loaded 1749 unique annotations


In [49]:
ns = NetworkSampler(gcms_peaks)

Creating adjacency structure. This might take some time...
Found 6232 (sparsity ratio = 0.00203726641475)


In [55]:
ns.initialise_sampler()

In [56]:
ns.multiple_network_sample(10000)

In [57]:
ns.dump_output(outstream=open('posteriors.txt','w'))